---

### 🎯 퀴즈: ReAct 패턴의 핵심 루프는?

ReAct Agent의 기본 실행 순서는?

**A)** Action → Thought → Observation
**B)** Thought → Action → Observation
**C)** Observation → Thought → Action


In [ ]:
# 여기를 수정하세요! 👇
# TODO: 정답을 선택하세요

my_answer = ""  # "A", "B", "C"

if my_answer == "B":
    print("✅ 정답!")
elif my_answer:
    print("❌ 다시 생각해보세요!")
else:
    print("💡 힌트를 보고 다시 시도하세요")

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답: B**

**해설:**
- **B) ✅ Thought (생각) → Action (행동) → Observation (관찰)**

**ReAct 루프:**
```
1. Thought: LLM이 무엇을 할지 생각
2. Action: 도구 실행
3. Observation: 결과 확인
→ 다시 1번으로 (목표 달성까지 반복)
```

**핵심:** "Reasoning (추론) + Acting (행동)"의 조합!

</details>

# 🤖 LLM 설정

ReAct Agent를 위한 LLM을 초기화합니다.

In [ ]:
!pip install -q transformers torch langchain-huggingface

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

print("🔄 EXAONE 모델 로딩 중...")

tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150, temperature=0.7, return_full_text=False)
llm = HuggingFacePipeline(pipeline=pipe)

print(f"✅ 모델 로딩 완료 ({'GPU' if torch.cuda.is_available() else 'CPU'} 사용)")

# 🛠️ 도구 정의

ReAct Agent가 사용할 도구들을 정의합니다.

In [ ]:
# 간단한 더미 도구 (실제로는 API 호출)
class DummyTool:
    def invoke(self, query):
        return f"[{self.__class__.__name__} 결과] {query}"

web_search = DummyTool()
doc_search = DummyTool()

web_search.__class__.__name__ = "WebSearch"
doc_search.__class__.__name__ = "DocSearch"

print("✅ 도구 준비 완료")
print("   - web_search: 웹 검색 시뮬레이션")
print("   - doc_search: 문서 검색 시뮬레이션")

# 5. 📊 ReAct State 정의

StateGraph의 핵심은 **State(상태)**입니다. 각 노드가 정보를 공유하는 공간이죠.

In [ ]:
# ReAct State 정의

from typing import TypedDict, List

class ReActState(TypedDict):
    question: str                  # 사용자 질문
    thoughts: List[str]            # Think 노드 판단 기록
    actions: List[str]             # Act 노드 도구 사용 기록
    observations: List[str]        # Act 노드 실행 결과
    iteration: int                 # 현재 반복 횟수
    max_iterations: int            # 최대 반복 (무한루프 방지)
    final_answer: str              # 최종 답변
    is_complete: bool              # 완료 여부

print("✅ ReActState 정의 완료")

# 6. 🎯 ReAct 노드 구현

ReAct Agent는 **4개의 노드**로 구성됩니다. 각 노드는 명확한 역할이 있습니다.

---

## 🔄 전체 흐름

```
사용자 질문 입력
    ↓
【Think】 LLM이 판단: "웹 검색 필요해!"
    ↓
【Act】 web_search.invoke() 실행 → 결과 수집
    ↓
【Observe】 "충분해?" → YES
    ↓
【Finalize】 LLM이 최종 답변 생성
    ↓
사용자에게 답변 전달
```

---

## 📋 각 노드 상세

### 1️⃣ Think 노드
**입력**: `state['question']`, `state['observations']` (이전 정보)  
**처리**: LLM에게 물어보고 전략 수립  
**출력**: `state['thoughts']` ← "TOOL:web_search" 같은 판단 저장

**핵심 코드**:
```python
llm_response = llm.invoke("어떤 도구?")
state['thoughts'].append(llm_response)
```

---

### 2️⃣ Act 노드
**입력**: `state['thoughts'][-1]` (방금 전 판단)  
**처리**: 판단에 따라 도구 **실제 실행**  
**출력**: `state['observations']` ← 도구 실행 결과 저장

**핵심 코드**:
```python
if "web_search" in last_thought:
    result = web_search.invoke(question)  # 🔥 여기서 실제 실행!
```

**중요**: 
- 여기서 Tavily API가 실제로 호출됩니다
- API 카운터가 차감되는 부분입니다

---

### 3️⃣ Observe 노드
**입력**: `state['observations'][-1]` (방금 수집한 정보)  
**처리**: "충분해?" 판단  
**출력**: `state['is_complete']` ← True면 완료, False면 다시 Think

**핵심 로직**:
```python
if 충분:
    state['is_complete'] = True  → Finalize로
else:
    state['iteration'] += 1      → Think로
```

---

### 4️⃣ Finalize 노드
**입력**: `state['observations']` (모든 수집 정보)  
**처리**: LLM이 최종 답변 생성  
**출력**: `state['final_answer']` ← 사용자에게 보여줄 답변

**핵심 코드**:
```python
prompt = f"질문: {question}
정보: {observations}
답변:"
final_answer = llm.invoke(prompt)
```

---

## 💡 왜 이렇게 나눴나요?

### 장점 1: **명확한 책임**
- Think는 "생각"만
- Act는 "행동"만
- Observe는 "판단"만
- Finalize는 "답변"만

### 장점 2: **디버깅 쉬움**
```
🤔 Think: OK
🛠️ Act: 여기서 에러! ← 문제 위치 파악 쉬움
```

### 장점 3: **재사용 가능**
- Think 로직만 바꾸고 싶다? → think_node만 수정
- 다른 도구 추가? → act_node만 수정

---

이제 코드를 봅시다! 👇 (리팩토링된 깔끔한 버전)

In [ ]:
# ReAct 노드 구현 (리팩토링 버전)

def think_node(state: ReActState) -> ReActState:
    """LLM이 어떤 도구를 사용할지 결정"""
    print(f"\n🤔 Think (반복 {state['iteration']}회)")
    
    question = state['question']
    history = "\n".join(state['observations'][-2:]) if state['observations'] else "없음"
    
    prompt = f"""질문: {question}
이전 관찰: {history}

선택지:
- "TOOL:web_search" (최신 정보, 뉴스, 2024년 이후)
- "TOOL:document_search" (AI, LangGraph, ReAct 내부 문서)
- "ANSWER" (충분한 정보, 바로 답변)

선택:"""
    
    llm_response = llm.invoke(prompt)
    print(f"   판단: {llm_response[:80]}...")
    
    state['thoughts'].append(llm_response)
    return state


def act_node(state: ReActState) -> ReActState:
    """선택한 도구를 실행"""
    print(f"🛠️ Act")
    
    last_thought = state['thoughts'][-1]
    
    if "web_search" in last_thought.lower():
        print(f"   → 웹 검색 실행")
        result = web_search.invoke(state['question'])
        observation = f"웹 검색: {str(result)[:200]}..."
        action = "web_search"
        
    elif "document_search" in last_thought.lower():
        print(f"   → 문서 검색 실행")
        result = doc_search.invoke(state['question'])
        observation = f"문서 검색: {str(result)[:200]}..."
        action = "document_search"
        
    else:
        print(f"   → 직접 답변")
        observation = "도구 불필요"
        action = "direct_answer"
    
    state['actions'].append(action)
    state['observations'].append(observation)
    return state


def observe_node(state: ReActState) -> ReActState:
    """🔥 LLM이 Observation을 보고 충분한지 판단 (create_react_agent 방식!)"""
    print(f"👀 Observe")
    
    # agent_scratchpad 구성 (create_react_agent 방식)
    history = ""
    for i in range(len(state['thoughts'])):
        if i < len(state['thoughts']):
            history += f"Thought: {state['thoughts'][i][:100]}...\n"
        if i < len(state['actions']):
            history += f"Action: {state['actions'][i]}\n"
        if i < len(state['observations']):
            history += f"Observation: {state['observations'][i][:200]}...\n\n"
    
    # 🔥 LLM에게 판단 요청!
    prompt = f"""질문: {state['question']}

지금까지의 과정:
{history}

위 결과를 보고 판단하세요:
- 충분한 정보가 모였다면: "FINISH"
- 더 검색이 필요하거나 다른 접근이 필요하면: "CONTINUE"

판단:"""
    
    llm_response = llm.invoke(prompt)
    print(f"   LLM 판단: {llm_response[:60]}...")
    
    # LLM 판단 + 하드코딩 백업 (둘 다 사용!)
    if "FINISH" in llm_response.upper() or "충분" in llm_response:
        print(f"   ✅ LLM이 완료 판단 → Finalize")
        state['is_complete'] = True
    elif state['iteration'] >= state['max_iterations']:
        print(f"   ⚠️ 최대 반복 도달 (하드코딩 백업) → Finalize")
        state['is_complete'] = True
    else:
        print(f"   ⏭️ LLM이 계속 판단 → Think")
        state['iteration'] += 1
    
    return state


def finalize_node(state: ReActState) -> ReActState:
    """최종 답변 생성"""
    print(f"📝 Finalize")
    
    observations = "\n".join(state['observations'])
    
    prompt = f"""질문: {state['question']}

수집 정보:
{observations}

위 정보로 답변 생성:"""
    
    final_answer = llm.invoke(prompt)
    state['final_answer'] = final_answer
    
    print(f"   ✅ 답변 완성 ({len(final_answer)} 글자)")
    return state


print("✅ 노드 정의 완료")
print("🔥 Observe 노드: LLM이 판단 + 하드코딩 백업 (create_react_agent 방식)")

# 7. 🔗 StateGraph로 ReAct Agent 구성

이제 노드들을 연결하여 ReAct 순환 구조를 만듭니다!

### 흐름도
```
START → Think → Act → Observe → 완료?
                         ↓          |
                         ← ← ← ← ← ← (No)
                         ↓
                        (Yes)
                         ↓
                     Finalize → END
```

---

### 🎯 실습: Calculator Tool 만들기

ReAct Agent가 사용할 계산기 도구를 만들어보세요!

**요구사항:**
- 도구 이름: "calculator"
- 기능: 두 숫자를 더하기
- 파라미터: a (숫자), b (숫자)

아래 셀에서 직접 작성해보세요.

In [ ]:
# 여기를 수정하세요! 👇
# TODO: calculator_tool 함수를 완성하세요

def calculator_tool(a: float, b: float) -> str:
    '''두 숫자를 더하는 도구'''
    # 여기에 덧셈 구현! 👇
    pass  # 이 줄을 지우고 작성하세요

# 테스트
try:
    result = calculator_tool(3, 5)
    if result == "8" or result == 8:
        print("✅ 정답!")
        print(f"   3 + 5 = {result}")
    else:
        print(f"❌ 결과가 이상해요: {result}")
except:
    print("💡 힌트: return str(a + b)")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
def calculator_tool(a: float, b: float) -> str:
    '''두 숫자를 더하는 도구'''
    result = a + b
    return str(result)
```

**Tool 레지스트리에 등록:**
```python
from langchain_core.tools import tool

@tool
def calculator(a: float, b: float) -> str:
    '''두 숫자를 더합니다'''
    return str(a + b)
```

**핵심:** Tool은 Agent가 호출할 수 있는 함수입니다!

</details>

---

### 🎯 실습 3: 조건부 라우팅 함수 만들기

ReAct Agent의 핵심은 **Observe 후 어디로 갈지 결정**하는 것입니다!

**요구사항:**
- `route_after_observe(state)` 함수 작성
- `is_complete`가 True면 → "finalize" 리턴
- `iteration >= max_iterations`면 → "finalize" 리턴
- 그 외 → "think" 리턴

아래 셀에서 직접 작성해보세요!

In [ ]:
def route_after_observe(state: ReActState) -> str:
    '''Observe 후 다음 노드를 결정하는 라우팅 함수'''
    # 여기를 수정하세요! 👇
    # is_complete나 max_iterations 도달 시 "finalize" 리턴
    # 그 외는 "think" 리턴
    pass  # 이 줄을 지우고 작성하세요

# 테스트
test_state_1 = {
    "question": "테스트",
    "thoughts": [],
    "actions": [],
    "observations": [],
    "iteration": 2,
    "max_iterations": 5,
    "final_answer": "",
    "is_complete": True  # 완료 상태
}

test_state_2 = {
    "question": "테스트",
    "thoughts": [],
    "actions": [],
    "observations": [],
    "iteration": 6,
    "max_iterations": 5,  # 최대 반복 초과
    "final_answer": "",
    "is_complete": False
}

test_state_3 = {
    "question": "테스트",
    "thoughts": [],
    "actions": [],
    "observations": [],
    "iteration": 2,
    "max_iterations": 5,  # 아직 반복 가능
    "final_answer": "",
    "is_complete": False
}

try:
    r1 = route_after_observe(test_state_1)
    r2 = route_after_observe(test_state_2)
    r3 = route_after_observe(test_state_3)
    
    if r1 == "finalize" and r2 == "finalize" and r3 == "think":
        print("✅ 정답!")
        print(f"   완료 상태 → {r1}")
        print(f"   최대 반복 도달 → {r2}")
        print(f"   계속 진행 → {r3}")
    else:
        print(f"❌ 결과가 이상해요: {r1}, {r2}, {r3}")
        print(f"   기대값: finalize, finalize, think")
except:
    print("💡 힌트: if state['is_complete'] or state['iteration'] >= state['max_iterations']")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
def route_after_observe(state: ReActState) -> str:
    '''Observe 후 다음 노드를 결정하는 라우팅 함수'''
    if state['is_complete'] or state['iteration'] >= state['max_iterations']:
        return "finalize"
    else:
        return "think"
```

**핵심 개념:**

1. **조건부 라우팅**: StateGraph에서 동적으로 다음 노드를 선택
2. **2가지 종료 조건**:
   - `is_complete = True`: LLM이 충분하다고 판단
   - `iteration >= max_iterations`: 최대 반복 도달 (무한루프 방지)

3. **add_conditional_edges 사용**:
```python
graph.add_conditional_edges(
    "observe",                    # 출발 노드
    route_after_observe,          # 라우팅 함수
    {
        "think": "think",         # 계속 진행
        "finalize": "finalize"    # 완료
    }
)
```

**실무 활용:**
- 에러 발생 시 재시도 vs 종료 판단
- 데이터 품질에 따라 추가 처리 vs 스킵
- 사용자 피드백에 따라 분기

</details>

In [ ]:
!pip install langgraph

In [ ]:
# StateGraph 구성

from langgraph.graph import StateGraph, START, END

# 1. Graph 생성
graph = StateGraph(ReActState)

# 2. 노드 추가
graph.add_node("think", think_node)
graph.add_node("act", act_node)
graph.add_node("observe", observe_node)
graph.add_node("finalize", finalize_node)

# 3. 기본 흐름
graph.add_edge(START, "think")
graph.add_edge("think", "act")
graph.add_edge("act", "observe")

# 4. 조건부 라우팅 (핵심!)
def route_after_observe(state: ReActState) -> str:
    """Observe 후 어디로 갈지 결정"""
    if state['is_complete'] or state['iteration'] >= state['max_iterations']:
        return "finalize"
    else:
        return "think"

graph.add_conditional_edges(
    "observe",
    route_after_observe,
    {"think": "think", "finalize": "finalize"}
)

graph.add_edge("finalize", END)

# 5. 컴파일
react_agent = graph.compile()

print("✅ StateGraph 완성!")
print("   Think → Act → Observe → (조건) → Think 또는 Finalize")

---

### 🎯 실습 4: 조건부 엣지 이해하기

방금 본 `add_conditional_edges()`의 동작 방식을 확인해봅시다.

**질문:** 다음 코드에서 `route_function`이 "option_a"를 리턴하면 어떤 노드로 이동할까요?

```python
graph.add_conditional_edges(
    "decision_node",
    route_function,
    {
        "option_a": "node_x",
        "option_b": "node_y",
        "option_c": "node_z"
    }
)
```

**A)** decision_node → node_x  
**B)** decision_node → node_y  
**C)** decision_node → route_function

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 정답을 선택하세요

my_answer = ""  # "A", "B", "C"

if my_answer == "A":
    print("✅ 정답!")
    print("   route_function이 'option_a'를 리턴")
    print("   → 매핑에서 'option_a': 'node_x' 찾음")
    print("   → node_x로 이동!")
elif my_answer:
    print("❌ 다시 생각해보세요!")
    print("💡 힌트: 딕셔너리에서 'option_a' 키의 값은?")
else:
    print("💡 힌트: route_function의 리턴값으로 딕셔너리를 조회합니다")

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답: A** decision_node → node_x

**해설:**

```python
graph.add_conditional_edges(
    "decision_node",           # 1. 출발 노드
    route_function,            # 2. 라우팅 함수 실행
    {
        "option_a": "node_x",  # 3. 리턴값으로 딕셔너리 조회
        "option_b": "node_y",
        "option_c": "node_z"
    }
)
```

**실행 흐름:**
1. `decision_node` 실행 완료
2. `route_function(state)` 호출 → "option_a" 리턴
3. 딕셔너리에서 `"option_a"` 키 찾기 → `"node_x"` 값
4. `node_x`로 이동!

**핵심:**
- 라우팅 함수는 **문자열**을 리턴
- 그 문자열을 **딕셔너리 키**로 사용
- 매핑된 **값(노드 이름)**으로 이동

**예시:**
```python
def route_function(state):
    if state['score'] > 80:
        return "high"    # → "node_advanced"로
    else:
        return "low"     # → "node_basic"로

graph.add_conditional_edges(
    "check_score",
    route_function,
    {
        "high": "node_advanced",
        "low": "node_basic"
    }
)
```

</details>

# 8. 🧪 ReAct Agent 테스트

이제 우리가 만든 ReAct Agent를 테스트해봅시다!

In [ ]:
# 테스트 함수

def test_react_agent(question: str, max_iterations: int = 3):
    """ReAct Agent 테스트"""
    
    print(f"\n{'='*60}")
    print(f"👤 질문: {question}")
    print(f"{'='*60}")
    
    # 초기 상태
    initial_state = ReActState(
        question=question,
        thoughts=[],
        actions=[],
        observations=[],
        iteration=1,
        max_iterations=max_iterations,
        final_answer="",
        is_complete=False
    )
    
    # 실행
    result = react_agent.invoke(initial_state)
    
    # 결과
    print(f"\n{'='*60}")
    print(f"🤖 최종 답변:")
    print(f"{'='*60}")
    print(result['final_answer'])
    print(f"\n📊 통계: 반복 {result['iteration']-1}회, 도구 {len(result['actions'])}개")
    
    return result

print("✅ 테스트 함수 준비 완료")

## 테스트 1: 문서 검색 (ReAct 패턴)

In [ ]:
# 테스트 1: 문서 검색

test_react_agent("ReAct 패턴이 무엇인지 설명해주세요")

## 테스트 2: 웹 검색 (최신 정보)

In [ ]:
# 테스트 2: 웹 검색

test_react_agent("2024년 AI 기술 동향을 알려주세요")

# 7. 🎮 직접 해보세요!

## 자유 테스트

아래에서 `test_react_agent("질문")` 형태로 다양한 질문을 해보세요!

### 추천 질문들
```python
# 문서 검색 질문
test_react_agent("LangGraph는 무엇인가요?")

# 웹 검색 질문  
test_react_agent("오늘 날씨는 어때요?")

# 일반 대화
test_react_agent("프로그래밍 공부 팁을 알려주세요")
```

In [ ]:
# 자유 테스트 공간 (stream_graph 방식)

print("🎮 자유롭게 테스트해보세요!")
print("💡 예시:")
print('test_react_agent("질문을 여기에 입력하세요")')
print()

# 여기서 테스트해보세요! 👇

# 추천 테스트 질문들:
# test_react_agent("LangGraph가 무엇인가요?")  # 문서 검색
# test_react_agent("오늘 날씨는 어때요?")        # 웹 검색  
# test_react_agent("안녕하세요!")               # 일반 대화

print("🌟 stream_graph 방식으로 실시간 스트리밍을 확인하세요!")

---

### 🎯 실습 5: 커스텀 도구 통합하기

이제 새로운 도구를 ReAct Agent에 추가해봅시다!

**시나리오:** 문자열 길이를 세는 `string_length` 도구 추가

**요구사항:**
1. `string_length_tool(text: str) -> str` 함수 작성
   - 입력 문자열의 길이를 리턴
   - 예: "안녕하세요" → "5"

2. `act_node`에 도구 추가
   - "string_length" 키워드 감지
   - 도구 실행 후 결과 저장

아래 셀에서 직접 작성해보세요!

In [ ]:
# 1단계: 도구 함수 작성
def string_length_tool(text: str) -> str:
    '''문자열의 길이를 반환하는 도구'''
    # 여기를 수정하세요! 👇
    pass  # 이 줄을 지우고 작성하세요

# 2단계: act_node에 통합
def enhanced_act_node(state: ReActState) -> ReActState:
    '''도구를 실행하는 노드 (string_length 추가)'''
    last_thought = state['thoughts'][-1] if state['thoughts'] else ""
    
    if "web_search" in last_thought.lower():
        result = "웹 검색 결과 (시뮬레이션)"
        observation = f"웹 검색: {result}"
        action = "web_search"
        
    elif "string_length" in last_thought.lower():
        # 여기를 수정하세요! 👇
        # string_length_tool 호출하고 결과 저장
        pass  # 이 줄을 지우고 작성하세요
        
    else:
        observation = "도구 불필요"
        action = "direct_answer"
    
    state['actions'].append(action)
    state['observations'].append(observation)
    return state

# 테스트
try:
    length_result = string_length_tool("안녕하세요")
    if length_result == "5" or length_result == 5:
        print("✅ 1단계 통과: 도구 함수 정상 작동")
        
        # act_node 테스트
        test_state = ReActState(
            question="'LangGraph'의 길이는?",
            thoughts=["TOOL:string_length 사용"],
            actions=[],
            observations=[],
            iteration=1,
            max_iterations=3,
            final_answer="",
            is_complete=False
        )
        
        result_state = enhanced_act_node(test_state)
        if "string_length" in result_state['actions']:
            print("✅ 2단계 통과: act_node 통합 성공!")
            print(f"   결과: {result_state['observations'][-1]}")
        else:
            print("❌ 2단계 실패: act_node에서 string_length 실행 안됨")
    else:
        print(f"❌ 1단계 실패: 길이가 {length_result} (기대값: 5)")
except Exception as e:
    print(f"💡 힌트: {str(e)}")
    print("   1단계: return str(len(text))")
    print("   2단계: result = string_length_tool(state['question'])")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
# 1단계: 도구 함수 작성
def string_length_tool(text: str) -> str:
    '''문자열의 길이를 반환하는 도구'''
    return str(len(text))

# 2단계: act_node에 통합
def enhanced_act_node(state: ReActState) -> ReActState:
    '''도구를 실행하는 노드 (string_length 추가)'''
    last_thought = state['thoughts'][-1] if state['thoughts'] else ""
    
    if "web_search" in last_thought.lower():
        result = "웹 검색 결과 (시뮬레이션)"
        observation = f"웹 검색: {result}"
        action = "web_search"
        
    elif "string_length" in last_thought.lower():
        # 질문에서 문자열 추출 (간단히 전체 질문 사용)
        text = state['question']
        result = string_length_tool(text)
        observation = f"문자열 길이: {result}"
        action = "string_length"
        
    else:
        observation = "도구 불필요"
        action = "direct_answer"
    
    state['actions'].append(action)
    state['observations'].append(observation)
    return state
```

**핵심 개념:**

1. **도구 = 함수**: 입력 → 처리 → 출력
2. **act_node에서 분기**: 키워드 감지 → 해당 도구 실행
3. **결과 저장**: observations에 기록

**실무 확장 예시:**

```python
# 더 많은 도구 추가
def act_node(state):
    thought = state['thoughts'][-1]
    
    if "web_search" in thought:
        result = tavily_search.invoke(query)
    elif "calculator" in thought:
        result = calculator_tool.invoke(expression)
    elif "database" in thought:
        result = db_query_tool.invoke(sql)
    elif "translation" in thought:
        result = translate_tool.invoke(text, target_lang)
    else:
        result = "No tool needed"
    
    return {"observations": [result]}
```

**@tool 데코레이터 활용:**
```python
from langchain_core.tools import tool

@tool
def string_length(text: str) -> str:
    '''문자열의 길이를 반환합니다'''
    return str(len(text))

# 자동으로 도구 이름, 설명, 파라미터가 정의됨
```

**도구 레지스트리 패턴:**
```python
TOOLS = {
    "web_search": tavily_search,
    "calculator": calculator_tool,
    "string_length": string_length_tool
}

def act_node(state):
    tool_name = extract_tool_name(state['thoughts'][-1])
    if tool_name in TOOLS:
        result = TOOLS[tool_name].invoke(state['question'])
    return {"observations": [result]}
```

</details>

# 💡 우리 구현 vs 실제 create_react_agent 비교

## ❓ 핵심 질문: "둘 다 ReAct인데 왜 다른가요?"

**답변**: ReAct **패턴**은 같고, **종료 판단 방식**도 같습니다! 차이는 **도구 호출 방식**입니다.

---

## ✅ 같은 점

### 1. ReAct 패턴
둘 다 **Think → Act → Observe** 순환 구조를 따릅니다.

```
질문 → 생각 → 도구 사용 → 결과 관찰 → 충분? 
           ↑                            ↓ 아니오
           └────────────────────────────┘
                                        ↓ 예
                                      답변
```

### 2. 종료 판단 방식
**둘 다 동일합니다!**

| 종료 조건 | create_react_agent | 우리 구현 | 
|-----------|-------------------|----------|
| 1차 판단 | ✅ LLM이 "Final Answer:" | ✅ LLM이 "FINISH" |
| 2차 백업 | ✅ max_iterations (하드코딩) | ✅ max_iterations (하드코딩) |

**핵심**: 
- 둘 다 LLM한테 먼저 물어봄
- LLM 못 믿으니 하드코딩 백업 있음
- 완전히 동일한 방식!

---

## 🎯 핵심 차이점: 도구 호출 방식

| create_react_agent | 우리 구현 |
|-------------------|----------|
| bind_tools() 자동 | 텍스트 파싱 + 직접 호출 |

### create_react_agent 방식

```python
# 1. 도구를 LLM에 등록
model = ChatOpenAI(model="gpt-4")
model_with_tools = model.bind_tools(tools)  # 🔥 자동화!

# 2. LLM이 자동으로 tool_calls JSON 생성
response = model_with_tools.invoke("날씨 알려줘")
# response.tool_calls = [{"name": "web_search", "args": {...}}]

# 3. tools 노드가 자동 실행
for tool_call in response.tool_calls:
    result = execute_tool(tool_call)
```

**문제**: HuggingFace 모델은 `bind_tools()` 지원 안 함!

---

### 우리 구현 방식

```python
# 1. LLM에게 "어떤 도구?" 물어봄
llm_response = llm.invoke("어떤 도구를 사용할까?")
# → "TOOL:web_search 사용하세요" (텍스트)

# 2. 우리가 직접 파싱
if "web_search" in llm_response:
    result = web_search.invoke(question)  # 🔥 직접 호출!

# 3. LLM에게 "충분해?" 물어봄
decision = llm.invoke("충분해?")
if "FINISH" in decision:
    # 완료
```

**장점**: 어떤 LLM이든 작동!

---

## 📊 상세 비교표

| 항목 | create_react_agent | 우리 구현 |
|------|-------------------|----------|
| **패턴** | ✅ ReAct | ✅ ReAct |
| **노드 수** | 2개 (agent, tools) | 4개 (think, act, observe, finalize) |
| **도구 호출** | bind_tools() 자동 | 텍스트 파싱 직접 |
| **종료 판단** | ✅ LLM + 하드코딩 | ✅ LLM + 하드코딩 |
| **모델 지원** | OpenAI/Anthropic만 | 모든 LLM |
| **투명성** | ❌ 블랙박스 | ✅ 모든 단계 확인 |

---

## 💡 핵심 깨달음

### 1. 종료 판단은 둘 다 같다!

```python
# create_react_agent든 우리든:
1. LLM이 먼저 판단 (충분해?)
2. max_iterations 도달 시 강제 종료 (하드코딩 백업)
```

**왜 하드코딩이 필요?**
- LLM을 100% 못 믿음
- 무한루프 방지
- 비용 제어

---

### 2. 진짜 차이는 도구 호출!

```python
# create_react_agent: 자동화
model.bind_tools(tools)  # → tool_calls JSON 자동 생성
# 👍 편하지만 특정 모델만 지원

# 우리: 수동
if "web_search" in llm_response:  # → 텍스트 파싱
# 👍 모든 모델 지원, 완전 제어
```

---

### 3. 노드 개수는 중요하지 않다!

- create_react_agent: 2개 노드 (agent, tools)
- 우리: 4개 노드 (think, act, observe, finalize)

**차이점**: 우리가 더 명시적으로 나눴을 뿐, 패턴은 동일!

---

## ⚖️ 언제 무엇을 써야 하나?

### create_react_agent 추천 👍

```
✅ OpenAI/Anthropic API 사용
✅ 빠른 프로토타입
✅ 간단한 도구
✅ 자동화 선호
```

---

### 우리 구현 추천 👍

```
✅ 오픈소스 모델 (EXAONE, Phi, Llama)
✅ Kaggle/Colab 환경
✅ 학습 목적 (원리 이해)
✅ 완전한 제어 필요
✅ 디버깅/모니터링 중요
```

---

## 🎓 학생들에게 이렇게 설명하세요!

### Q1: "뭐가 다른가요?"
**A**: 도구 호출 방식만 다릅니다!
- create_react_agent: `bind_tools()` 자동화
- 우리: 텍스트 파싱 수동

**종료 판단은 둘 다 같습니다!**
- LLM이 먼저 판단
- 하드코딩 백업 (max_iterations)

---

### Q2: "뭐가 더 좋은가요?"
**A**: 상황에 따라!
- OpenAI 쓰면 → create_react_agent
- 오픈소스 쓰면 → 우리 구현

---

### Q3: "같은 ReAct인데 왜 다른가요?"
**A**: 패턴은 100% 같습니다!
- Think → Act → Observe 순환
- LLM이 종료 판단 + 하드코딩 백업
- **다른 건 도구 호출 방식뿐!**

---

### Q4: "반복 횟수는 LLM이 정하나요?"
**A**: 반은 맞고 반은 틀립니다!

```python
# 1차 판단: LLM (이상적)
llm.invoke("충분해?")
# → "FINISH" or "CONTINUE"

# 2차 백업: 하드코딩 (안전장치)
if iteration >= 15:  # 강제 종료!
    break
```

**둘 다 씁니다!** LLM 못 믿으니까요.

---

## 🔥 핵심 정리

### 우리가 만든 것

```python
def observe_node(state):
    # 🔥 LLM에게 물어봄!
    decision = llm.invoke("충분해?")
    
    if "FINISH" in decision:  # LLM 판단
        return "finalize"
    elif iteration >= max_iterations:  # 하드코딩 백업
        return "finalize"
    else:
        return "think"
```

**이게 create_react_agent와 완전히 동일합니다!**

---

### 결론

**"우리 구현 = create_react_agent의 원리"**

차이점:
- 도구 호출: 자동 vs 수동
- 노드 개수: 2개 vs 4개 (명시성 차이)

같은 점:
- ✅ ReAct 패턴
- ✅ LLM 종료 판단
- ✅ 하드코딩 백업
- ✅ Think/Act/Observe 순환

**🌟 패턴과 원리는 동일합니다!**

# 🎓 정리 - StateGraph로 만든 ReAct Agent

## 🌟 완성한 것

### ✅ 핵심 구현
1. **ReActState**: 모든 정보를 담는 상태 객체
2. **Think 노드**: LLM으로 어떤 도구 사용할지 결정
3. **Act 노드**: 도구를 **직접 실행** 🔥
4. **Observe 노드**: 결과 분석 및 완료 판단
5. **StateGraph**: 노드들을 순환 구조로 연결

### 🔥 핵심 코드
```python
# 도구 직접 호출!
def act_node(state):
    if "web_search" in state['thoughts'][-1]:
        result = web_search.invoke(question)  # 🔥 직접 실행!
    return state

# StateGraph 순환
react_graph.add_conditional_edges(
    "observe",
    should_continue,  # 완료? → finalize, 아님? → think
)
```

### 💡 ReAct Agent의 장점 (다시 한번!)

#### 1. **반복적 추론**
- 한 번에 답을 못 찾아도 다시 시도
- 여러 단계를 거쳐 정확한 답변

#### 2. **투명한 과정**
- Think/Act/Observe 각 단계 확인 가능
- 디버깅과 개선이 쉬움

#### 3. **도구 조합**
- 웹 검색 + 문서 검색 + ...
- 상황에 맞게 최적의 도구 선택

#### 4. **실패 복구**
- 최대 반복 횟수로 무한루프 방지
- 다른 전략으로 재시도 가능

### 🚀 다음 단계

이제 여러분은:
- ✅ StateGraph로 복잡한 워크플로우 설계 가능
- ✅ ReAct 패턴의 원리 이해
- ✅ AI Agent에 도구 통합하는 방법 습득

더 나아가기:
- 더 많은 도구 추가 (계산기, 데이터베이스, API 등)
- 더 복잡한 라우팅 로직 구현
- 멀티 에이전트 시스템 구축

### 🎯 핵심 메시지

**"create_react_agent 같은 편의 함수도 좋지만,  
StateGraph로 직접 구현하면 완전한 제어와 이해가 가능합니다!"**

이게 바로 LangGraph의 진짜 힘입니다. 🔥